<a href="https://colab.research.google.com/github/sayyed-uoft/TSSA/blob/main/Vector_Institute_Optimization_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vector Institute - TSSA Intro to AI 

### Thank you for joining Day 2 of the Vector Institute, 'Intro to AI' workshop series.

If you have any questions or if you would like to learn more about this program, contact: learn@vectorinstitute.ai

# Case Study 3: Optimazation using Clustering


In this notebook, we will use k-Maens and Size Constrained k-Means to optiomize agent-client assignment. 


## Problem
We have “n” clients and “m” agents (n >> m). We want to assign the clients to the agents, so each agent travels as less as possible to visit their own clients. It means the clients assigned to an agent should be very close to each other.


## Solution
A simple solution is to cluster the “m” clients to “n” clusters based on their closeness. We can use K-means clustering (“K = m” in this case).

## Importing the required packages


In [ ]:
# Import python packages
import numpy as np
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

# Make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

## Simulating the Problem (Synthetic Data)

n = 10,000

m = k = 5

In [ ]:
# A package to generate data blobs
from sklearn.datasets import make_blobs

# Choose some random centers for the clusters
blob_centers = np.array(
    [[ 0.2,  2.3],
     [-1.5 ,  2.3],
     [-2.8,  1.8],
     [-2.8,  2.8],
     [-2.8,  1.3]])

# Set how spread each cluster is
blob_std = np.array([0.4, 0.3, 0.1, 0.1, 0.1])

# Generate data
X, y = make_blobs(n_samples=2000, centers=blob_centers,
                  cluster_std=blob_std, random_state=0)

# Plot data

In [ ]:
# A function to plot clusters
def plot_clusters(X, y=None):
    plt.scatter(X[:, 0], X[:, 1], c=y, s=1)
    plt.xlabel("$x_1$", fontsize=14)
    plt.ylabel("$x_2$", fontsize=14, rotation=0)

# Set plot size
plt.figure(figsize=(10, 6))

# Plot the clusters
plot_clusters(X)
plt.show()

# Clustering using regular K-Means

In [ ]:
# Import K-Mean clustering tool
from sklearn.cluster import KMeans

# Set the number of clusters
k = 5

# Predict the clusters
kmeans = KMeans(n_clusters=k, random_state=0)
y_pred = kmeans.fit_predict(X)

### View the predictions

In [ ]:
y_pred

### View the cluster centers

In [ ]:
kmeans.cluster_centers_

## Plot the clusters and their boundaries

In [ ]:
# A function to plot all points
def plot_data(X):
    plt.plot(X[:, 0], X[:, 1], 'k.', markersize=2)

# A function to show the center of the clusters
def plot_centroids(centroids, weights=None, circle_color='w', cross_color='k'):
    if weights is not None:
        centroids = centroids[weights > weights.max() / 10]
    plt.scatter(centroids[:, 0], centroids[:, 1],
                marker='o', s=35, linewidths=8,
                color=circle_color, zorder=10, alpha=0.9)
    plt.scatter(centroids[:, 0], centroids[:, 1],
                marker='x', s=2, linewidths=12,
                color=cross_color, zorder=11, alpha=1)

# A function to plot decision boundaries
def plot_decision_boundaries(clusterer, X, resolution=1000, show_centroids=True,
                             show_xlabels=True, show_ylabels=True):
    mins = X.min(axis=0) - 0.1
    maxs = X.max(axis=0) + 0.1
    xx, yy = np.meshgrid(np.linspace(mins[0], maxs[0], resolution),
                         np.linspace(mins[1], maxs[1], resolution))
    Z = clusterer.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.contourf(Z, extent=(mins[0], maxs[0], mins[1], maxs[1]),
                cmap="Pastel2")
    plt.contour(Z, extent=(mins[0], maxs[0], mins[1], maxs[1]),
                linewidths=1, colors='k')
    plot_data(X)
    if show_centroids:
        plot_centroids(clusterer.cluster_centers_)

    if show_xlabels:
        plt.xlabel("$x_1$", fontsize=14)
    else:
        plt.tick_params(labelbottom=False)
    if show_ylabels:
        plt.ylabel("$x_2$", fontsize=14, rotation=0)
    else:
        plt.tick_params(labelleft=False)

In [ ]:
# plot cluster centers and their boundaries
plt.figure(figsize=(10, 6))
plot_decision_boundaries(kmeans, X)
plt.show()

## Unbalanced Clusters
With k-Means algorithm, we can get unequal clusters. That can be an issue for agent-client assignment. Let's generate some unbalanced clusters:


In [ ]:
# The same cluster centers
blob_centers = np.array(
    [[ 0.2,  2.3],
     [-1.5 ,  2.3],
     [-2.8,  1.8],
     [-2.8,  2.8],
     [-2.8,  1.3]])

# The second blob is set to be more spread
blob_std = np.array([0.4, 0.5, 0.1, 0.1, 0.1])

# The seciond cluster has a lot more clients
X, y = make_blobs(n_samples=[1500, 4000, 1500, 1500, 1500], 
                  centers=blob_centers, cluster_std=blob_std, random_state=0)

### Plot the new clusters

In [ ]:
# plot the clusters
plt.figure(figsize=(10, 6))
plot_clusters(X)
plt.show()

### Clsuter the new data using regular k-Means and show the cluster borders

In [ ]:
# predict the clusters
y_pred = kmeans.fit_predict(X)

# plot the cluster boundaries 
plt.figure(figsize=(10, 6))
plot_decision_boundaries(kmeans, X)
plt.show()

### Check the clusters' balance
The clusters are changed. Let's look at the number of points in each cluster:

In [ ]:
np.unique(y_pred, return_counts=True)

The numbers are not the same

## Size Constrained K-Means
Let's use a constrained k-Means that we can set min and max number of clients in each cluster. In this experiment, we will set them the same.


## Installing the required library
Let's start by installing the constrained k-Mean library.

In [ ]:
!pip install k-means-constrained

## Predict the clusters with the size constrain

In [ ]:
# Import constrained k-Means package
from k_means_constrained import KMeansConstrained

# Predict the new clusters
kmeans = KMeansConstrained(n_clusters=k, size_min=2000, size_max=2000, random_state=0)
y_pred = kmeans.fit_predict(X)

In [ ]:
# View the new cluster centers
kmeans.cluster_centers_

## Plot the clusters

In [ ]:
# Import color map utility
import matplotlib.cm as cm

# plot the clusters with different colors
colors = cm.viridis(np.linspace(0, 1, k))
plt.figure(figsize=(10, 6))
for cluster in range(k):
  X_temp = X[y_pred == cluster]
  plt.plot(X_temp[:, 0], X_temp[:, 1], 'o', color=colors[cluster], markersize=2)
plot_centroids(kmeans.cluster_centers_)

Let's check if the clusters have equal number of clients

In [ ]:
np.unique(y_pred, return_counts=True)

They are all the same

## Suggested improvement

We were able to guarantee the equal number of clients assigned to each agent and as much as possible they are close to each other. Is it still fair? Is the distance traveled by each agent the same? What can we do to make it fair?

# Contact Information

Congratulations, you have completed the tutorial for Day 2 of the Vector Institute 'Intro to AI' program! Thank you for your time and attention.


*   Instructor: Sayyed Nezhadi 
*   Program Director: Shingai Manjengwa 
*   Contact: learn@vectorinstitute.ai

Never stop learning!